In [26]:
import pandas as pd
import wandb
api = wandb.Api()
from consts import BATCHSIZE, all_collections
from consts import cnn_group_data_extra_features_no_leak as cnn_group_data

databases = ["raw","meters","exponential","all_collections","all_spaces"]
spaces = ["indoor","garage","outdoor"]

model_dict:dict = {}

# Project is specified by <entity/project-name>
for group in cnn_group_data.keys():
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    print(f"{group} : {len(runs)}")
    for run in runs:
        model_dict[group] = model_dict.get(group,[])
        model_dict[group].append(run.name)


CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_reto_grande_garage_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features_no_leak : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features_no_leak : 3


In [27]:
display(model_dict)

{'CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features_no_leak': ['reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5',
  'reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5',
  'reto_grande_outdoor_extra_features_no_leak_run2_depth3_model2'],
 'CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features_no_leak': ['reto_grande_indoor_extra_features_no_leak_run0_depth1_model7',
  'reto_grande_indoor_extra_features_no_leak_run1_depth2_model0',
  'reto_grande_indoor_extra_features_no_leak_run2_depth2_model4'],
 'CNN_DATA_ANALYSIS_reto_grande_garage_extra_features_no_leak': ['reto_grande_garage_extra_features_no_leak_run0_depth4_model0',
  'reto_grande_garage_extra_features_no_leak_run1_depth1_model2',
  'reto_grande_garage_extra_features_no_leak_run2_depth1_model0'],
 'CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features_no_leak': ['all_spaces_reto_grande_extra_features_no_leak_run0_depth4_model6',
  'all_spaces_reto_grande_extra_features_no_leak_run1_depth4_model0',
  '

In [28]:
import os
from model_evaluation import evaluate_model_on_collections
from feature_lists import DATASET_TO_FEATURE

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

results = {}
for group in cnn_group_data:
    results[group] = []


for current_group,all_runs in model_dict.items():

    for current_run in all_runs:
        current_group_data      = cnn_group_data[current_group]
        current_model_storage   = current_group_data["model_storage"]
        current_database        = current_group_data["database"]
        current_model_path      = f"{current_model_storage}/{current_run}.pt"
        feature_list            = DATASET_TO_FEATURE[current_database]
        relative_coeficient     = current_group_data["relative_coeficient"]
        model_output_name       = current_run
        val_collection          = current_group_data["validation_collections"]

        for collections_name, current_collections in val_collection.items(): 
            
            if not os.path.exists(current_model_path):
                print(f"Model doesnt exist {model_output_name} with path {current_model_path}")
                continue
            
            try:
                mae, mse, rmse, n = evaluate_model_on_collections(
                    current_model_path, current_collections, current_database, BATCHSIZE, None, feature_list
                )
            except Exception as e:
                print(f"Failed Processing {model_output_name} with path {current_model_path}")
                print(e)
                continue
            mae_final = mae * relative_coeficient
            mse_final = mse * relative_coeficient
            rmse_final = rmse * relative_coeficient
            print(f"{model_output_name} | Collections : {collections_name} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
            results[current_group].append((model_output_name, collections_name, mae_final, mse_final, rmse_final))




reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5 | Collections : reto_grande_outdoor | N=19550 | MAE=3.206652 | MSE=0.644391 | RMSE=4.540981
reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5 | Collections : reto_grande_indoor | N=12243 | MAE=3.641216 | MSE=0.544170 | RMSE=4.172941
reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5 | Collections : reto_grande_garage | N=22621 | MAE=8.417013 | MSE=3.478773 | RMSE=10.550865
reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5 | Collections : all_reto_grande | N=54414 | MAE=5.470480 | MSE=1.800152 | RMSE=7.589786
reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5 | Collections : reto_grande_outdoor | N=19550 | MAE=3.095156 | MSE=0.617328 | RMSE=4.444603
reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5 | Collections : reto_grande_indoor | N=12243 | MAE=4.819287 | MSE=1.029788 | RMSE=5.740490
reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5 | Collections : reto

In [29]:
import pandas as pd
for group, rows in results.items():

    df = pd.DataFrame(rows, columns=["model","collections", "mae", "mse", "rmse"])
    print(f"GROUP : {group}")
    styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
    display(styled)

GROUP : CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features_no_leak


,model,collections,mae,mse,rmse
0,reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5,reto_grande_outdoor,3.206652,0.644391,4.540981
1,reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5,reto_grande_indoor,3.641216,0.544170,4.172941
2,reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5,reto_grande_garage,8.417013,3.478773,10.550865
3,reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5,all_reto_grande,5.470480,1.800152,7.589786
4,reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5,reto_grande_outdoor,3.095156,0.617328,4.444603
5,reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5,reto_grande_indoor,4.819287,1.029788,5.740490
6,reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5,reto_grande_garage,10.704245,4.993721,12.641166
7,reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5,all_reto_grande,6.646333,2.529485,8.996863
8,reto_grande_outdoor_extra_features_no_leak_run2_depth3_model2,reto_grande_outdoor,3.028756,0.596021,4.367229
9,reto_grande_outdoor_extra_features_no_leak_run2_depth3_model2,reto_grande_indoor,7.616405,2.393020,8.750808


GROUP : CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features_no_leak


,model,collections,mae,mse,rmse
0,reto_grande_indoor_extra_features_no_leak_run0_depth1_model7,reto_grande_outdoor,9.725799,4.494877,11.993167
1,reto_grande_indoor_extra_features_no_leak_run0_depth1_model7,reto_grande_indoor,0.000037,0.000000,0.000037
2,reto_grande_indoor_extra_features_no_leak_run0_depth1_model7,reto_grande_garage,9.872559,4.643528,12.189868
3,reto_grande_indoor_extra_features_no_leak_run0_depth1_model7,all_reto_grande,7.598540,3.545339,10.651331
4,reto_grande_indoor_extra_features_no_leak_run1_depth2_model0,reto_grande_outdoor,9.725791,4.494869,11.993157
5,reto_grande_indoor_extra_features_no_leak_run1_depth2_model0,reto_grande_indoor,0.000000,0.000000,0.000000
6,reto_grande_indoor_extra_features_no_leak_run1_depth2_model0,reto_grande_garage,9.872549,4.643521,12.189859
7,reto_grande_indoor_extra_features_no_leak_run1_depth2_model0,all_reto_grande,7.598525,3.545333,10.651322
8,reto_grande_indoor_extra_features_no_leak_run2_depth2_model4,reto_grande_outdoor,9.725780,4.494859,11.993143
9,reto_grande_indoor_extra_features_no_leak_run2_depth2_model4,reto_grande_indoor,0.000032,0.000000,0.000033


GROUP : CNN_DATA_ANALYSIS_reto_grande_garage_extra_features_no_leak


...

,model,collections,mae,mse,rmse
0,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_garage,4.110398,0.961544,5.547019
1,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_outdoor,9.121450,4.039515,11.369454
2,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_indoor,0.337511,0.006843,0.467961
3,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_garage_outdoor,7.519861,3.055761,9.888597
4,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_garage_indoor,3.095988,0.704855,4.749250
5,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_indoor_outdoor,7.827569,3.445500,10.500285
6,all_collections_indoor_garage_extra_features_no_leak_run0_depth3_model3,all_collections_garage_outdoor_indoor,6.764470,2.735097,9.355378
7,all_collections_indoor_garage_extra_features_no_leak_run1_depth2_model5,all_collections_garage,2.750545,0.583402,4.320748
8,all_collections_indoor_garage_extra_features_no_leak_run1_depth2_model5,all_collections_outdoor,9.322426,4.212417,11.610225
9,all_collections_indoor_garage_extra_features_no_leak_run1_depth2_model5,all_collections_indoor,0.109530,0.000798,0.159822


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features_no_leak


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_garage,3.549756,0.782914,5.005322
1,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_outdoor,3.996923,0.919545,5.424521
2,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_indoor,15.974659,13.223971,20.571025
3,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_garage_outdoor,3.854003,0.875876,5.294151
4,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_garage_indoor,6.890420,4.127921,11.493192
5,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_indoor_outdoor,5.761252,2.731996,9.350073
6,all_collections_garage_outdoor_extra_features_no_leak_run0_depth3_model0,all_collections_garage_outdoor_indoor,5.128772,2.174565,8.341827
7,all_collections_garage_outdoor_extra_features_no_leak_run1_depth4_model3,all_collections_garage,3.702522,0.835470,5.170592
8,all_collections_garage_outdoor_extra_features_no_leak_run1_depth4_model3,all_collections_outdoor,3.893293,0.879341,5.304612
9,all_collections_garage_outdoor_extra_features_no_leak_run1_depth4_model3,all_collections_indoor,16.021988,10.962003,18.729231


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features_no_leak


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_garage,4.106583,0.956702,5.533033
1,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_outdoor,4.424611,1.069516,5.850173
2,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_indoor,0.382168,0.010675,0.584466
3,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_garage_outdoor,4.322966,1.033459,5.750713
4,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_garage_indoor,3.105206,0.702345,4.740785
5,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_indoor_outdoor,3.829156,0.913548,5.406805
6,all_collections_garage_outdoor_indoor_extra_features_no_leak_run0_depth3_model5,all_collections_garage_outdoor_indoor,3.908499,0.925890,5.443204
7,all_collections_garage_outdoor_indoor_extra_features_no_leak_run1_depth1_model2,all_collections_garage,3.090517,0.658013,4.588727
8,all_collections_garage_outdoor_indoor_extra_features_no_leak_run1_depth1_model2,all_collections_outdoor,3.616219,0.802957,5.068988
9,all_collections_garage_outdoor_indoor_extra_features_no_leak_run1_depth1_model2,all_collections_indoor,0.152743,0.001533,0.221491


In [30]:
import re
import pandas as pd
import matplotlib.pyplot as plt


combined_rows = []

for group, rows in results.items():
    
    dataset = max((d for d in databases if d in group), key=len, default=None)
    space = max((s for s in spaces if s in group), key=len, default=None)
    
    for current_row in rows:
        print(current_row)
        (model_output_name, collections_name, mae_final, mse_final, rmse_final) = current_row
        combined_rows.append((group,collections_name,mae_final,mse_final,rmse_final))

print(combined_rows)

('reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5', 'reto_grande_outdoor', 3.206652390511749, 0.6443910286432641, 4.540981492649409)
('reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5', 'reto_grande_indoor', 3.641215612634592, 0.5441698351346871, 4.172940776516003)
('reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5', 'reto_grande_garage', 8.417013317872241, 3.478773227690133, 10.550864575288808)
('reto_grande_outdoor_extra_features_no_leak_run0_depth4_model5', 'all_reto_grande', 5.4704804227857515, 1.8001516077656659, 7.589785994907979)
('reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5', 'reto_grande_outdoor', 3.095155828968643, 0.6173280982654114, 4.444603373136141)
('reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5', 'reto_grande_indoor', 4.8192868341780715, 1.029788401657914, 5.7404902972701946)
('reto_grande_outdoor_extra_features_no_leak_run1_depth4_model5', 'reto_grande_garage', 10.704244936583903, 4.9937210666274

In [31]:
import pandas as pd

singles = {
    "CNN_DATA_ANALYSIS_reto_grande_garage_exponential"          : "reto_grande_garage_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_exponential"          : "reto_grande_indoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential"         : "reto_grande_outdoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_garage_raw"                  : "reto_grande_garage_raw",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_raw"                  : "reto_grande_indoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_raw"                 : "reto_grande_outdoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_garage_meters"               : "reto_grande_garage_meters",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_meters"               : "reto_grande_indoor_meters",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_meters"              : "reto_grande_outdoor_meters",
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande"                  : "all_spaces_reto_grande",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features"      : "reto_grande_outdoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features"       : "reto_grande_indoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_garage_extra_features"       : "reto_grande_garage_extra_features", 
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features"   : "all_spaces_reto_grande_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features_no_leak"      : "reto_grande_outdoor_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features_no_leak"       : "reto_grande_indoor_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_reto_grande_garage_extra_features_no_leak"       : "reto_grande_garage_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features_no_leak"   : "all_spaces_reto_grande_extra_features_no_leak", 
}

spaces = {
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor"                      : "all_coll_garage_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage"                       : "all_coll_indoor_garage",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor"                      : "all_coll_indoor_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor"               : "all_coll_garage_outdoor_indoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features"       : "all_coll_indoor_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features"        : "all_coll_indoor_garage_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features"       : "all_coll_garage_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features": "all_coll_garage_outdoor_indoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features_no_leak"       : "all_coll_indoor_outdoor_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features_no_leak"        : "all_coll_indoor_garage_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features_no_leak"       : "all_coll_garage_outdoor_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features_no_leak": "all_coll_garage_outdoor_indoor_extra_features_no_leak",
}

spaces_datasets = []
singles_datasets = []

for row in combined_rows:
    (group,collections_name,mae_final,mse_final,rmse_final) = row
    if group in spaces:
        group_defnition = spaces[group]
        spaces_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    elif group in singles:
        group_defnition = singles[group]
        singles_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    else:
        print(f"ERROR : {group}")

print(spaces_datasets)
print(singles_datasets)


[('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_garage', 9.23222441084603, 4.204221607267669, 11.598926305161413), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_outdoor', 4.186753133526333, 0.9771993917785308, 5.591992537272648), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_indoor', 0.5618550013400162, 0.01800839020079751, 0.7591234987968165), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_garage_outdoor', 5.799342545854027, 2.0085920384512854, 8.017165660658456), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_garage_indoor', 6.901036447790646, 3.0786813118268244, 9.925613430839345), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_indoor_outdoor', 3.652802922725493, 0.8359098305184045, 5.171954618574001), ('all_coll_indoor_outdoor_extra_features_no_leak', 'all_collections_garage_outdoor_indoor', 5.248499140975492, 1.7992359482989642, 7.587855451019534), ('all_

In [32]:
import pandas as pd

# rows: list[tuple] like (dataset, collections, mae, mse, rmse)
def make_wide_by_collection(rows, *,
                            caption="Modelos - erros médios por coleção",
                            label="tab:model_metrics_by_collection"):
    # Build DataFrame
    df = pd.DataFrame(rows, columns=["dataset", "collections", "mae", "mse", "rmse"])

    if df.empty:
        print("No data provided.")
        return None, None, None

    # Robust pivot (handles duplicate (dataset, collections) by averaging)
    df_wide = (
        df.pivot_table(index="dataset",
                       columns="collections",
                       values=["mae", "mse", "rmse"],
                       aggfunc="mean")
          .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
          .sort_index(axis=1, level=[0, 1])        # nice column order
          .sort_index(axis=0)                      # sort datasets
    )

    # Optional: round displayed values only
    styled = (
        df_wide.style
              .format(precision=2, decimal=",")
              .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=caption,
        label=label,
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
    )

    return df, df_wide, (styled, latex.replace("_", "\\_"))


def latex_with_lines(df, *args, **kwargs):
    kwargs['column_format'] = '|'.join([''] + ['l'] * df.index.nlevels
                                            + ['r'] * df.shape[1] + [''])
    res = df.to_latex(*args, **kwargs)
    return res.replace('\\\\\n', '\\\\ \\midrule\n')

In [33]:
# Map collections labels
CNN_space_combinations = {
    "all_collections_garage"                : "garage",
    "all_collections_outdoor"               : "outdoor",
    "all_collections_indoor"                : "indoor",
    "all_collections_garage_outdoor"        : "garage_outdoor",
    "all_collections_garage_indoor"         : "garage_indoor",
    "all_collections_indoor_outdoor"        : "indoor_outdoor",
    "all_collections_garage_outdoor_indoor" : "garage_outdoor_indoor",
}

# Build DataFrame
df = pd.DataFrame(spaces_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])

# Robust pivot (handles duplicate (dataset, collections) by averaging)
df_wide = (
    df.pivot_table(index="dataset",
                    columns="collections",
                    values=["mae", "mse", "rmse"],
                    aggfunc="mean")
        .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
        .sort_index(axis=1, level=[0, 1])        # nice column order
        .sort_index(axis=0)                      # sort datasets
)

df_wide = df_wide.sort_values(by="dataset", key=lambda x: x.str.len())
df_wide = df_wide.transpose()

# Optional: round displayed values only
styled = (
    df_wide.style
            .format(precision=2, decimal=",")
            .background_gradient(cmap="RdYlGn_r")
)

latex = styled.to_latex(
    caption="Dataset Combination errors",
    label="tb:model_metrics_by_collection_spaces",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
)


display(styled)
print(latex.replace("_", "\\_"))




\begin{table}[H]
\caption{Dataset Combination errors}
\label{tb:model\_metrics\_by\_collection\_spaces}
\begin{tabular}{llrrrr}
\toprule
 & dataset & all\_coll\_indoor\_garage\_extra\_features\_no\_leak & all\_coll\_garage\_outdoor\_extra\_features\_no\_leak & all\_coll\_indoor\_outdoor\_extra\_features\_no\_leak & all\_coll\_garage\_outdoor\_indoor\_extra\_features\_no\_leak \\
collections &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{all\_collections\_garage} & mae & {\cellcolor[HTML]{A2D76A}} \color[HTML]{000000} 3,38 & {\cellcolor[HTML]{30A356}} \color[HTML]{F1F1F1} 3,59 & {\cellcolor[HTML]{F57245}} \color[HTML]{F1F1F1} 9,81 & {\cellcolor[HTML]{FEC877}} \color[HTML]{000000} 3,49 \\
 & mse & {\cellcolor[HTML]{108647}} \color[HTML]{F1F1F1} 0,76 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,80 & {\cellcolor[HTML]{D3EC87}} \color[HTML]{000000} 4,83 & {\cellcolor[HTML]{39A758}} \color[HTML]{F1F1F1} 0,77 \\
 & rmse & {\cellcolor[HTML]{E2F397}} \color[HTML]{000000} 4,89 & {\cellcolo

In [34]:
# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["model", "collections", "mae", "mse", "rmse"])

# Map collections labels
CNN_space_combinations = {
    "all_collections_garage"                : "garage",
    "all_collections_outdoor"               : "outdoor",
    "all_collections_indoor"                : "indoor",
    "all_collections_garage_outdoor"        : "garage_outdoor",
    "all_collections_garage_indoor"         : "garage_indoor",
    "all_collections_indoor_outdoor"        : "indoor_outdoor",
    "all_collections_garage_outdoor_indoor" : "garage_outdoor_indoor",
}
#df["collections"] = df["collections"].map(CNN_space_combinations)

# Ensure numeric cols are floats (in case they came as strings with commas)
num_cols = ["mae", "mse", "rmse"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Aggregate and create a MultiIndex (dataset -> collections)
df = (
    df.groupby(["model", "collections"], as_index=False)[num_cols]
      .mean()
      .sort_values(["model", "collections"])
      .set_index(["model", "collections"])   # <- multirow index for rows
)

# Styled display in notebook (keeps the MultiIndex)
styled = (
    df.style
      .format(precision=2, decimal=",")
      .background_gradient(cmap="RdYlGn_r", subset=num_cols)
)

display(styled)

latex = styled.to_latex(
    caption=f"{dataset} error",
    label=f"tb:cnn_all_datasets_{dataset}",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
    # multirow=True,  # uncomment if your pandas version exposes this arg on Styler.to_latex
)

print(latex.replace("_", "\\_"))


\begin{table}[H]
\caption{all\_collections error}
\label{tb:cnn\_all\_datasets\_all\_collections}
\begin{tabular}{llrrr}
\toprule
 &  & mae & mse & rmse \\
model & collections &  &  &  \\
\midrule
\multirow[c]{4}{*}{all\_spaces\_reto\_grande\_extra\_features\_no\_leak} & all\_reto\_grande & {\cellcolor[HTML]{7FC866}} \color[HTML]{000000} 2,48 & {\cellcolor[HTML]{16914D}} \color[HTML]{F1F1F1} 0,48 & {\cellcolor[HTML]{A7D96B}} \color[HTML]{000000} 3,90 \\
 & reto\_grande\_garage & {\cellcolor[HTML]{A2D76A}} \color[HTML]{000000} 3,04 & {\cellcolor[HTML]{1E9A51}} \color[HTML]{F1F1F1} 0,57 & {\cellcolor[HTML]{B7E075}} \color[HTML]{000000} 4,26 \\
 & reto\_grande\_indoor & {\cellcolor[HTML]{04703B}} \color[HTML]{F1F1F1} 0,17 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{04703B}} \color[HTML]{F1F1F1} 0,24 \\
 & reto\_grande\_outdoor & {\cellcolor[HTML]{AFDD70}} \color[HTML]{000000} 3,29 & {\cellcolor[HTML]{2AA054}} \color[HTML]{F1F1F1} 0,67 & {\cellcolor[HTML]{C3E

In [35]:



# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])
#display(df)

datasets = list(df["collections"].unique())
for dataset in datasets:
    print(f"Dataset: {dataset}")
    df_current = df.loc[df['collections'] == dataset]

    num_cols = ['mae', 'mse', 'rmse']
    df_current = (
        df_current.groupby('dataset', as_index=False)[num_cols]
        .mean()
    )
    df_current = df_current.sort_values('dataset')
    styled = (
        df_current.style
                .format(precision=2, decimal=",")
                .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=f"{dataset} error",
        label=f"tb:cnn_all_datasets_{dataset}",
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
        hrules=True,
    )
    

    display(styled)
    print(latex.replace("_", "\\_"))




Dataset: reto_grande_outdoor


,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features_no_leak,"3,29","0,67","4,61"
1,reto_grande_garage_extra_features_no_leak,"10,29","5,33","12,83"
2,reto_grande_indoor_extra_features_no_leak,"9,73","4,49","11,99"
3,reto_grande_outdoor_extra_features_no_leak,"3,11","0,62","4,45"


\begin{table}[H]
\caption{reto\_grande\_outdoor error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_outdoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features\_no\_leak & {\cellcolor[HTML]{06733D}} \color[HTML]{F1F1F1} 3,29 & {\cellcolor[HTML]{026C39}} \color[HTML]{F1F1F1} 0,67 & {\cellcolor[HTML]{04703B}} \color[HTML]{F1F1F1} 4,61 \\
1 & reto\_grande\_garage\_extra\_features\_no\_leak & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 10,29 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 5,33 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 12,83 \\
2 & reto\_grande\_indoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{CC2627}} \color[HTML]{F1F1F1} 9,73 & {\cellcolor[HTML]{ED5F3C}} \color[HTML]{F1F1F1} 4,49 & {\cellcolor[HTML]{D62F27}} \color[HTML]{F1F1F1} 11,99 \\
3 & reto\_grande\_outdoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 3,11 & {\cellcolor[HTML]{006837}} \color[HTML]{

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features_no_leak,"0,17","0,00","0,24"
1,reto_grande_garage_extra_features_no_leak,"7,70","2,85","9,26"
2,reto_grande_indoor_extra_features_no_leak,"0,00","0,00","0,00"
3,reto_grande_outdoor_extra_features_no_leak,"5,36","1,32","6,22"


\begin{table}[H]
\caption{reto\_grande\_indoor error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_indoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features\_no\_leak & {\cellcolor[HTML]{05713C}} \color[HTML]{F1F1F1} 0,17 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{06733D}} \color[HTML]{F1F1F1} 0,24 \\
1 & reto\_grande\_garage\_extra\_features\_no\_leak & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 7,70 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 2,85 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,26 \\
2 & reto\_grande\_indoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 \\
3 & reto\_grande\_outdoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{FDAF62}} \color[HTML]{000000} 5,36 & {\cellcolor[HTML]{F1F9AC}} \color[HTML]{00000

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features_no_leak,"3,04","0,57","4,26"
1,reto_grande_garage_extra_features_no_leak,"2,83","0,52","4,08"
2,reto_grande_indoor_extra_features_no_leak,"9,87","4,64","12,19"
3,reto_grande_outdoor_extra_features_no_leak,"9,56","4,27","11,66"


\begin{table}[H]
\caption{reto\_grande\_garage error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_garage}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features\_no\_leak & {\cellcolor[HTML]{07753E}} \color[HTML]{F1F1F1} 3,04 & {\cellcolor[HTML]{036E3A}} \color[HTML]{F1F1F1} 0,57 & {\cellcolor[HTML]{05713C}} \color[HTML]{F1F1F1} 4,26 \\
1 & reto\_grande\_garage\_extra\_features\_no\_leak & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 2,83 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,52 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 4,08 \\
2 & reto\_grande\_indoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,87 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 4,64 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 12,19 \\
3 & reto\_grande\_outdoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{BB1526}} \color[HTML]{F1F1F1} 9,56 & {\cellcolor[HTML]{D22B27}} \color[HTML]{F1F1

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features_no_leak,"2,48","0,48","3,90"
1,reto_grande_garage_extra_features_no_leak,"6,60","2,77","9,32"
2,reto_grande_indoor_extra_features_no_leak,"7,60","3,55","10,65"
3,reto_grande_outdoor_extra_features_no_leak,"6,30","2,30","8,54"


\begin{table}[H]
\caption{all\_reto\_grande error}
\label{tb:cnn\_all\_datasets\_all\_reto\_grande}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features\_no\_leak & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 2,48 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,48 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 3,90 \\
1 & reto\_grande\_garage\_extra\_features\_no\_leak & {\cellcolor[HTML]{F26841}} \color[HTML]{F1F1F1} 6,60 & {\cellcolor[HTML]{F98E52}} \color[HTML]{F1F1F1} 2,77 & {\cellcolor[HTML]{F36B42}} \color[HTML]{F1F1F1} 9,32 \\
2 & reto\_grande\_indoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 7,60 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 3,55 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 10,65 \\
3 & reto\_grande\_outdoor\_extra\_features\_no\_leak & {\cellcolor[HTML]{F99153}} \color[HTML]{000000} 6,30 & {\cellcolor[HTML]{FEE28F}} \color[HTML]{000000} 2,